# Computate Smart Cloud Builder

## About the open source GPL3 license and copyright for this product

Copyright © 2024 Computate Limited Liability Company in Utah, USA

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

ADDITIONAL TERMS

As stated in section 7. c) and e) of the GPL3 license, 
"you may supplement the terms of this License with terms," 
Computate has added the following additional terms to the license: 

  7 c) Prohibiting misrepresentation of the origin of that material, and
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version;

  7 e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks;

Please do not redistribute this course until you have built your own platform with these tools, 
separate from the computate.org platform, and reconfigure your fork of this repo to deploy 
your own platform instead of the computate.org platform. 

QUESTIONS

For questions about this open source license, please contact our public mailing list at computate@group.computate.org


## Switch to the Bash Kernel

To run this Jupyter Notebook in VSCode, you will need to switch to the Bash Kernel you installed earlier. 
It's a button in the top right corner of the notebook that says `Select Kernel`. 


## Upload default computate Solr configset
A Solr configset is a data schema for a search engine. It specifies the exact fields and their data types that are available in the search engine to use.

In [ ]:
oc exec pod/solr-0 -- \
  install -d /bitnami/solr/backup/
oc rsync solr/ pod/solr-0:/bitnami/solr/backup/
echo DONE

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
oc exec pod/solr-0 -- \
  bash -c 'curl -s -X POST -u "$SOLR_ADMIN_USERNAME:$SOLR_ADMIN_PASSWORD" \
      -H "Content-Type:application/octet-stream" \
      --data-binary @/bitnami/solr/backup/computate-configset.zip \
      "http://localhost:8983/solr/admin/configs?action=UPLOAD&name=computate&overwrite=true"'
echo DONE

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
oc exec pod/solr-0 -- \
  bash -c 'export SOLR_COLLECTION='"$SOLR_COLLECTION"' && SOLR_AUTH_TYPE=basic SOLR_AUTHENTICATION_OPTS="-Dbasicauth=$SOLR_ADMIN_USERNAME:$SOLR_ADMIN_PASSWORD" /opt/bitnami/solr/bin/solr create_collection -c $SOLR_COLLECTION -n computate'
echo DONE

## Run SQL create scripts with new PostgreSQL tables
When we create new Smart Data Models that are persisted in the PostgreSQL database, we need to make sure that our database has all the tables and fields required to persist the data. Run the commands below to rsync the `db-create.sql` script to the PostgreSQL pod. It will connect again and apply all of the database schema changes to the database. 

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
oc exec pod/postgresql-0 -- mkdir -p /bitnami/postgresql/sql/
oc rsync $SITE_SRC/src/main/resources/sql/ \
  pod/postgresql-0:/bitnami/postgresql/sql/
oc exec pod/postgresql-0 -- \
  env PGPASSWORD=$(oc get secret/postgres-pguser-$DATABASE_USERNAME -o jsonpath={.data.password} | base64 -d) \
  psql -U $DATABASE_USERNAME -d $DATABASE_DATABASE -f /bitnami/postgresql/sql/db-create.sql
echo DONE

# Deploy your site! 

At this point, you have built your new site, containerized it, 
and created the kustomization infrastructure as code to deploy it. 


We just need to kustomize the overlays with a few environment variables 
that include `$SITE_HOST_NAME` and `$SITE_BASE_URL`, which come from the 
vars.yaml file. 
Run the command below to override these values in the overlay files. 

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
replace '$SITE_HOST_NAME' "$SITE_HOST_NAME" -- $OVERLAY_SITE_PATH/deployment.yaml
replace '$SITE_BASE_URL' "$SITE_BASE_URL" -- $OVERLAY_SITE_PATH/deployment.yaml
replace '$SITE_HOST_NAME' "$SITE_HOST_NAME" -- $OVERLAY_SITE_PATH/route.yaml

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
oc apply -k $OVERLAY_SITE_PATH